# What is Qablet?
A **Qablet timetable** defines a financial product using a sequence of payments, choices and conditions. A valuation model implemented with a Qablet parser can value any contract, as long as the contract can be described using a Qablet Timetable.

## A Simple Contract

In this notebook we will learn to create the simplest possible financial contract as a qablet timetable, and price it with a deterministic model. We will use the following packages.

- `qablet_contracts` contains utilities to create qablet timetables for financial contracts.
- `qablet` contains a suite of models to evaluate qablet timetables.

To use these packages in your local environment, you will need `pip install qablet-contracts` and `pip install qablet-basic`. For complete API documentation see [qablet.github.io/qablet-contracts](https://qablet.github.io/qablet-contracts/) and [qablet-academy.github.io/intro/](https://qablet-academy.github.io/intro/).

Let us start with the imports.

In [ ]:
import numpy as np
import pandas as pd
from pyarrow import RecordBatch as rb
from datetime import datetime
from qablet_contracts.timetable import TS_EVENT_SCHEMA, py_to_ts
from qablet.base.fixed import FixedModel

## Create the contract

A contract is described by a list of events. We will create our first contract using a single event. An event has five properties:

- time
- op
- quantity
- unit
- track

The event below is an example of a `+` **op**, which pays **quantity** 100, at **time** 1.0. In later notebooks we will learn about tracks, units, and other ops.


In [ ]:
events = [
    {
        "time": datetime(2024, 12, 31),
        "op": "+",
        "quantity": 100.0,
        "unit": "USD",
        "track": "",
    },
]
timetable = {"events": rb.from_pylist(events, schema=TS_EVENT_SCHEMA)}
print(timetable["events"].to_pandas())

                       time op  quantity unit track
0 2024-12-31 00:00:00+00:00  +     100.0  USD      


The `events` component in the timetable is an [arrow recordbatch](https://arrow.apache.org/docs/format/Columnar.html#). Here we have created it using a list of dicts, but you could also create it using [pandas, polars, and other methods](https://qablet.github.io/qablet-contracts/specifications/timetable/#create-a-timetable).

## Create a Dataset to price the contract
For our simple contract, the following data is needed
- BASE. The name of the currency in which the contract will be valued.
- PRICING_TS. The timestamp (milliseconds) that we will price the contract as of.
- ASSETS. The forward or discount curve for all assets required for valuation. We will define the asset `USD` using an array of times (years) and zero rates (fraction).

`FixedModel` needs nothing else. Other models will reuire model specific parameters in the dataset, as we will see in other notebooks. For more details see [Dataset API](https://qablet-academy.github.io/intro/dataset/)

In [ ]:
# 4% rate from 0 to 5 years.
discount_data = ("ZERO_RATES", np.array([[5.0, 0.04]]))
dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(datetime(2023, 12, 31)).value,
    "ASSETS": {"USD": discount_data},
}

## Calculate Price

In [ ]:
model = FixedModel()
price, _ = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:   96.068415


## Contract with multiple events

A sequence of payments is represented by a sequence of `+` events on the same track. In this example we construct a three year bond, paying a 5% coupon semi-annually. 

In [ ]:
dates = pd.bdate_range(
    datetime(2023, 12, 31),
    datetime(2026, 12, 31),
    freq="2QE",
    inclusive="right",
)
# Interest Payments
events = [
    {
        "track": "",
        "time": dt,
        "op": "+",
        "quantity": 5 / 2,
        "unit": "USD",
    }
    for dt in dates
]

# Principal
events.append(
    {
        "track": "",
        "time": dates[-1],
        "op": "+",
        "quantity": 100.0,
        "unit": "USD",
    }
)

timetable = {
    "events": rb.from_pylist(events, schema=TS_EVENT_SCHEMA),
}
print(timetable["events"].to_pandas())

                       time op  quantity unit track
0 2024-06-30 00:00:00+00:00  +       2.5  USD      
1 2024-12-31 00:00:00+00:00  +       2.5  USD      
2 2025-06-30 00:00:00+00:00  +       2.5  USD      
3 2025-12-31 00:00:00+00:00  +       2.5  USD      
4 2026-06-30 00:00:00+00:00  +       2.5  USD      
5 2026-12-31 00:00:00+00:00  +       2.5  USD      
6 2026-12-31 00:00:00+00:00  +     100.0  USD      


We calculate the price as before.

In [ ]:
price, _ = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:  102.676020


## Learn more

These notebooks illustrate how to **create different contracts**, and price them using models in the qablet-basic package.

- [Vanilla Option](1_2_vanilla_option.ipynb)
- [Equity Cliquet](1_3_equity_cliquet.ipynb)
- [Vanilla Swaption](1_4_vanilla_swaption.ipynb)
- [Bermuda Swaption](1_5_bermuda_swaption.ipynb)

These notebooks illustrate how to **define your own monte-carlo model**, and use them to price qablet timetables.

- [Black Scholes](2_1_custom_mc.ipynb)
- [Two Asset Black Scholes](2_2_custom_two_asset.ipynb)


Next: [Vanilla Option](1_2_vanilla_option.ipynb)